# Notes # 

- Following reddit comment
    - 104, 52, 100, 97, 109, 65, 114, 100 is `h, 4, d, a, m, A, r, d` in ASCII
    - Conclude, probably want to use **Hadamard Gate(s)** on some of the qubits
- Useful docs from Qiskit
    - [Elementary Operations](https://qiskit-staging.mybluemix.net/documentation/terra/summary_of_quantum_operations.html)
- [The Reddit Conversation](https://www.reddit.com/r/may4quantum/comments/gd5b8p/congratulations/)
- Important conversion: H=X(pi)Y(pi/2)
- [QC StackExchange (general decomp.)](https://quantumcomputing.stackexchange.com/questions/8538/how-to-decompose-a-unitary-transform-into-two-level-unitary-matrices)
    - multiply *U* by *stuff* until you get the identity
- [StackOverflow (specifically Krauss-Cirac decomposition)](https://stackoverflow.com/questions/58198077/is-there-any-systematic-way-to-decompose-a-two-level-unitary-matrix-into-single)
- **NB** Might want to look up LU Decomp. packages in Python
- [Transpile Tutorial](https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/terra/advanced/4_transpiler_passes_and_passmanager.ipynb)

# Goal of challenge: #
## Guess the circuit that created U ##

In [1]:
#standard imports
from qiskit import QuantumCircuit, BasicAer, execute

In [2]:
# Useful additional packages
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from math import pi

In [3]:
#set up from the problem
qc = QuantumCircuit(4)

In [4]:
#set up a backend
backend = BasicAer.get_backend("unitary_simulator")

# What is your plan? #

- know that we are trying to guess the circuit that produced unitary
- know from QM-SE problem that essentially doing LU decomp on unitary
- idea is use a std. alg. to do the LU decomp for me
- then inspect L and U
- finally match L and U to the gates in `elementary operations`
    - CAN ONLY USE U3 and CNOT
    - need to figure out a way of picking out $\theta$, $\phi$, $\lambda$

In [7]:
import numpy as np
import quantum_decomp

In [8]:
U = np.matrix([[-0.21338835+0.33838835j, -0.14016504-0.08838835j,
         0.21338835-0.08838835j,  0.03661165+0.08838835j,
         0.08838835-0.03661165j, -0.08838835-0.21338835j,
        -0.08838835+0.14016504j,  0.33838835+0.21338835j,
         0.21338835-0.08838835j,  0.03661165+0.08838835j,
         0.39016504+0.08838835j, -0.03661165+0.16161165j,
         0.16161165+0.03661165j,  0.08838835-0.39016504j,
         0.08838835-0.03661165j, -0.08838835-0.21338835j],
       [-0.14016504-0.08838835j, -0.21338835+0.33838835j,
         0.03661165+0.08838835j,  0.21338835-0.08838835j,
        -0.08838835-0.21338835j,  0.08838835-0.03661165j,
         0.33838835+0.21338835j, -0.08838835+0.14016504j,
         0.03661165+0.08838835j,  0.21338835-0.08838835j,
        -0.03661165+0.16161165j,  0.39016504+0.08838835j,
         0.08838835-0.39016504j,  0.16161165+0.03661165j,
        -0.08838835-0.21338835j,  0.08838835-0.03661165j],
       [ 0.21338835-0.08838835j,  0.03661165+0.08838835j,
        -0.21338835+0.33838835j, -0.14016504-0.08838835j,
        -0.08838835+0.14016504j,  0.33838835+0.21338835j,
         0.08838835-0.03661165j, -0.08838835-0.21338835j,
         0.39016504+0.08838835j, -0.03661165+0.16161165j,
         0.21338835-0.08838835j,  0.03661165+0.08838835j,
         0.08838835-0.03661165j, -0.08838835-0.21338835j,
         0.16161165+0.03661165j,  0.08838835-0.39016504j],
       [ 0.03661165+0.08838835j,  0.21338835-0.08838835j,
        -0.14016504-0.08838835j, -0.21338835+0.33838835j,
         0.33838835+0.21338835j, -0.08838835+0.14016504j,
        -0.08838835-0.21338835j,  0.08838835-0.03661165j,
        -0.03661165+0.16161165j,  0.39016504+0.08838835j,
         0.03661165+0.08838835j,  0.21338835-0.08838835j,
        -0.08838835-0.21338835j,  0.08838835-0.03661165j,
         0.08838835-0.39016504j,  0.16161165+0.03661165j],
       [ 0.08838835-0.03661165j, -0.08838835-0.21338835j,
        -0.08838835+0.14016504j,  0.33838835+0.21338835j,
        -0.21338835+0.33838835j, -0.14016504-0.08838835j,
         0.21338835-0.08838835j,  0.03661165+0.08838835j,
         0.16161165+0.03661165j,  0.08838835-0.39016504j,
         0.08838835-0.03661165j, -0.08838835-0.21338835j,
         0.21338835-0.08838835j,  0.03661165+0.08838835j,
         0.39016504+0.08838835j, -0.03661165+0.16161165j],
       [-0.08838835-0.21338835j,  0.08838835-0.03661165j,
         0.33838835+0.21338835j, -0.08838835+0.14016504j,
        -0.14016504-0.08838835j, -0.21338835+0.33838835j,
         0.03661165+0.08838835j,  0.21338835-0.08838835j,
         0.08838835-0.39016504j,  0.16161165+0.03661165j,
        -0.08838835-0.21338835j,  0.08838835-0.03661165j,
         0.03661165+0.08838835j,  0.21338835-0.08838835j,
        -0.03661165+0.16161165j,  0.39016504+0.08838835j],
       [-0.08838835+0.14016504j,  0.33838835+0.21338835j,
         0.08838835-0.03661165j, -0.08838835-0.21338835j,
         0.21338835-0.08838835j,  0.03661165+0.08838835j,
        -0.21338835+0.33838835j, -0.14016504-0.08838835j,
         0.08838835-0.03661165j, -0.08838835-0.21338835j,
         0.16161165+0.03661165j,  0.08838835-0.39016504j,
         0.39016504+0.08838835j, -0.03661165+0.16161165j,
         0.21338835-0.08838835j,  0.03661165+0.08838835j],
       [ 0.33838835+0.21338835j, -0.08838835+0.14016504j,
        -0.08838835-0.21338835j,  0.08838835-0.03661165j,
         0.03661165+0.08838835j,  0.21338835-0.08838835j,
        -0.14016504-0.08838835j, -0.21338835+0.33838835j,
        -0.08838835-0.21338835j,  0.08838835-0.03661165j,
         0.08838835-0.39016504j,  0.16161165+0.03661165j,
        -0.03661165+0.16161165j,  0.39016504+0.08838835j,
         0.03661165+0.08838835j,  0.21338835-0.08838835j],
       [ 0.21338835-0.08838835j,  0.03661165+0.08838835j,
         0.39016504+0.08838835j, -0.03661165+0.16161165j,
         0.16161165+0.03661165j,  0.08838835-0.39016504j,
         0.08838835-0.03661165j, -0.08838835-0.21338835j,
        -0.21338835+0.33838835j, -0.14016504-0.08838835j,
         0.21338835-0.08838835j,  0.03661165+0.08838835j,
         0.08838835-0.03661165j, -0.08838835-0.21338835j,
        -0.08838835+0.14016504j,  0.33838835+0.21338835j],
       [ 0.03661165+0.08838835j,  0.21338835-0.08838835j,
        -0.03661165+0.16161165j,  0.39016504+0.08838835j,
         0.08838835-0.39016504j,  0.16161165+0.03661165j,
        -0.08838835-0.21338835j,  0.08838835-0.03661165j,
        -0.14016504-0.08838835j, -0.21338835+0.33838835j,
         0.03661165+0.08838835j,  0.21338835-0.08838835j,
        -0.08838835-0.21338835j,  0.08838835-0.03661165j,
         0.33838835+0.21338835j, -0.08838835+0.14016504j],
       [ 0.39016504+0.08838835j, -0.03661165+0.16161165j,
         0.21338835-0.08838835j,  0.03661165+0.08838835j,
         0.08838835-0.03661165j, -0.08838835-0.21338835j,
         0.16161165+0.03661165j,  0.08838835-0.39016504j,
         0.21338835-0.08838835j,  0.03661165+0.08838835j,
        -0.21338835+0.33838835j, -0.14016504-0.08838835j,
        -0.08838835+0.14016504j,  0.33838835+0.21338835j,
         0.08838835-0.03661165j, -0.08838835-0.21338835j],
       [-0.03661165+0.16161165j,  0.39016504+0.08838835j,
         0.03661165+0.08838835j,  0.21338835-0.08838835j,
        -0.08838835-0.21338835j,  0.08838835-0.03661165j,
         0.08838835-0.39016504j,  0.16161165+0.03661165j,
         0.03661165+0.08838835j,  0.21338835-0.08838835j,
        -0.14016504-0.08838835j, -0.21338835+0.33838835j,
         0.33838835+0.21338835j, -0.08838835+0.14016504j,
        -0.08838835-0.21338835j,  0.08838835-0.03661165j],
       [ 0.16161165+0.03661165j,  0.08838835-0.39016504j,
         0.08838835-0.03661165j, -0.08838835-0.21338835j,
         0.21338835-0.08838835j,  0.03661165+0.08838835j,
         0.39016504+0.08838835j, -0.03661165+0.16161165j,
         0.08838835-0.03661165j, -0.08838835-0.21338835j,
        -0.08838835+0.14016504j,  0.33838835+0.21338835j,
        -0.21338835+0.33838835j, -0.14016504-0.08838835j,
         0.21338835-0.08838835j,  0.03661165+0.08838835j],
       [ 0.08838835-0.39016504j,  0.16161165+0.03661165j,
        -0.08838835-0.21338835j,  0.08838835-0.03661165j,
         0.03661165+0.08838835j,  0.21338835-0.08838835j,
        -0.03661165+0.16161165j,  0.39016504+0.08838835j,
        -0.08838835-0.21338835j,  0.08838835-0.03661165j,
         0.33838835+0.21338835j, -0.08838835+0.14016504j,
        -0.14016504-0.08838835j, -0.21338835+0.33838835j,
         0.03661165+0.08838835j,  0.21338835-0.08838835j],
       [ 0.08838835-0.03661165j, -0.08838835-0.21338835j,
         0.16161165+0.03661165j,  0.08838835-0.39016504j,
         0.39016504+0.08838835j, -0.03661165+0.16161165j,
         0.21338835-0.08838835j,  0.03661165+0.08838835j,
        -0.08838835+0.14016504j,  0.33838835+0.21338835j,
         0.08838835-0.03661165j, -0.08838835-0.21338835j,
         0.21338835-0.08838835j,  0.03661165+0.08838835j,
        -0.21338835+0.33838835j, -0.14016504-0.08838835j],
       [-0.08838835-0.21338835j,  0.08838835-0.03661165j,
         0.08838835-0.39016504j,  0.16161165+0.03661165j,
        -0.03661165+0.16161165j,  0.39016504+0.08838835j,
         0.03661165+0.08838835j,  0.21338835-0.08838835j,
         0.33838835+0.21338835j, -0.08838835+0.14016504j,
        -0.08838835-0.21338835j,  0.08838835-0.03661165j,
         0.03661165+0.08838835j,  0.21338835-0.08838835j,
        -0.14016504-0.08838835j, -0.21338835+0.33838835j]])

In [9]:
print(quantum_decomp.matrix_to_qsharp(U))

AttributeError: module 'quantum_decomp' has no attribute 'matrix_to_qsharp'

# NEW PLAN #

- Try controlled H to sqish down the number of total gates
    - I'm thinking something like X RY and CNOT
- Test using CHad etc. to see if you can approx the unitary
- If above approximates, then translate into U3 and CNOT
- last hour #helpmehadamard